Convert the flood damages from NatCat to 2005 ppp-corrected values (i.e. inflationsbereinigt und kaufkraftbereinigt)

Note: We do not need to take the MC events into account (those with missing ISOs), they are already accounted for with individual entries for each country (same 'MR Number')


In [6]:
from __future__ import division
import sys
import netCDF4
import dimarray as da
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.spatial.distance
from scipy import spatial
from functools import partial
from time import time
import datetime
from netCDF4 import Dataset
import matplotlib.legend as legend
import math

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

#######################
#Natcat dataset
path='/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/1980-2016_Masterdatei_NatCat_worldwide_no-pw.xlsx'
natcat=pd.read_excel(path,usecols=np.arange(1,49))

#Iso codes
path = '/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/ISO_codes.csv'
num=pd.read_csv(path)

#EmDat Dataset
path='/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/20170715_emdat.xlsx'
emdat=pd.read_excel(path)
natcat.head()

,Event ID,MR Number,MR ID,Period,Year,Begin,End,Event Text,Event Family \n(A = Geophysical events; \nB = Meteorological events; \nC = Hydrological events; \nD = Climatological/Extreme temperature events),Subevent,...,"estimated insured losses for statistics\n(EUR m, original values)","official overall losses for publication\n(EUR m, adjusted to inflation to 2015 values based on national CPI)","official insured losses for publication\n(EUR m, adjusted to inflation to 2015 values based on national CPI)","estimated overall losses for statistics \n(EUR m, adjusted to inflation to 2015 values based on national CPI)","estimated insured losses for statistics \n(EUR m, adjusted to inflation to 2015 values based on national CPI)",Fatalities,Catastrophe Class,Geocoded Place,Longitude,Latitude
0,15344,MR198001D066,MR198001D066_ESP,January 1980- August 1982,1980,1980-01-01,1982-08-31,Drought,D,dr:Drought,...,0.000000,3254.264431,0.000000,3254.264431,0.000000,0,4,ESP: Murcia,-1.485377,38.002378
1,14808,MR198001A033,MR198001A033_PRT,1.1.1980,1980,1980-01-01,1980-01-01,Earthquake,A,gs:Ground shaking/Earthquake,...,0.000000,106.776091,0.000000,106.776091,0.000000,57,2,Terceira,-27.220577,38.721642
2,10844,MR198001B003,MR198001B003_AUS,January 1980,1980,1980-01-01,1980-01-01,Cyclone Amy,B,tc:Tropical cyclone,...,3.006398,10.621318,7.965989,10.621318,7.965989,0,2,Perth,115.861000,-31.950000
3,12372,MR198001B005,MR198001B005_AFG,Januar 1980,1980,1980-01-01,1980-01-31,Snowstorms,B,bs:Blizzard/Snowstorm,...,0.000000,0.000000,0.000000,0.028378,0.000000,0,0,Afghanestan,65.000000,33.000000
4,13659,MR198001B022,MR198001B022_GRC,1.-3.1.1980,1980,1980-01-01,1980-01-03,Snowstorm,B,bs:Blizzard/Snowstorm,...,0.000000,0.000000,0.000000,0.237854,0.000000,5,1,Preveza,20.667000,39.167000


In [7]:
#Extract relevant data from nat_cat: type= general flood

natcat_short=natcat.iloc[:,[4,7,8,9,13,12,11,21,20,25,29,33,46,47]]
cols=['year','event','type','subtype','ISO','country','Continent','tot_loss','ins_loss','tot_loss_GDP','tot_loss_GCP','tot_loss_CPI','Longitude', 'Latitude']
natcat_short.columns=cols
natcat_hydro=natcat_short.loc[natcat_short['subtype']=='gf:General flood']
natcat_short

# more information on columns
# 4: "Year"
# 7: "Event text"
# 8: "Event familiy: (A= Geophysical events; B=Meterological events; C=Hydrological events; D=Climatological/Extreme temperature events)
# 9: "Subevent", e.,g. "General flood"
# 13: "Country ISO3"
# 12: "Countray Name"
# 11: "Continent"
# 21: "estimated overall losses for statistics (US$ m, original values)"
# 20: "official insured losses for publication (US$ m, original values)"
# 25: "estimated overall losses for statistics (US$, normalized to 2016 values based on national GDP)"
# 29: "estimated overall losses for statistics (US$, normalized to 2016 values based on GCP method)"
# 33: "estimated overall losses for statistics (US$, adjusted to inflation to 2016 values based on national CPI)"

,year,event,type,subtype,ISO,country,Continent,tot_loss,ins_loss,tot_loss_GDP,tot_loss_GCP,tot_loss_CPI,Longitude,Latitude
0,1980,Drought,D,dr:Drought,ESP,Spain,EUROP,1500.00,0.0,7776.285421,10338.403680,3589.576050,-1.485377,38.002378
1,1980,Earthquake,A,gs:Ground shaking/Earthquake,PRT,Portugal,EUROP,40.00,0.0,258.086525,293.759557,118.148812,-27.220577,38.721642
2,1980,Cyclone Amy,B,tc:Tropical cyclone,AUS,Australia,OCEAN,4.00,3.0,30.554906,40.463144,10.901736,115.861000,-31.950000
3,1980,Snowstorms,B,bs:Blizzard/Snowstorm,AFG,Afghanistan,ASIA,0.01,0.0,0.050349,0.058855,0.029127,65.000000,33.000000
4,1980,Snowstorm,B,bs:Blizzard/Snowstorm,GRC,Greece,EUROP,0.10,0.0,0.328316,0.367235,0.263188,20.667000,39.167000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30557,2016,"Flash flood, landslide",C,ff:Flash flood,ECU,Ecuador,AMER,2.00,0.0,2.000000,2.000000,2.000000,-77.987099,-1.482849
30558,2016,Winter storm,B,ws:Winter storm,NaN,"Canada, USA",AMER,6.00,0.0,NaN,NaN,NaN,-57.304400,50.649770
30559,2016,Winter storm,B,ws:Winter storm,CAN,Canada,AMER,5.00,0.0,5.000000,5.000000,5.000000,-58.365333,49.101857
30560,2016,"Flash flood, landslide",C,ff:Flash flood,BOL,Bolivia,AMER,1.50,0.0,1.500000,1.500000,1.500000,-67.400000,-16.983333


In [8]:
#Get the relevant columns in analogue to nat_cat
#EMDAT has some 'flood' events without 'subtype' (subtype = '--')
#This might be the reason for an underestimation of damages compared to NatCat

#
emdat_short=emdat.iloc[:,[0,1,2,3,4,11]]
cols=['year','ISO','country','type','subtype','tot_loss']
emdat_short.columns=cols

#Some Riverine Floods have 'NAN' as 'type, this is corrected for here
for i in range(0,len(emdat_short)):
    if emdat_short.loc[i,'subtype'] == 'Riverine flood':
                     emdat_short.loc[i,'type'] = 'Flood'

#Some Flood events have '--' as subtype, as emdat was underestimating damages before, 
#we're keeping those cases in now. 
#Check if this is COOL.

emdat_hydro=emdat_short.loc[emdat_short['type'] == 'Flood']
emdat_hydro=emdat_hydro.loc[emdat_hydro['subtype'] != 'Coastal flood']
emdat_hydro=emdat_hydro.loc[emdat_hydro['subtype'] != 'Flash flood']
emdat_hydro=emdat_hydro.loc[emdat_hydro['subtype'] != 'Convective storm'] #Doesn't matter, only two events without damage

/home/insauer/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
# How many losses in EMdat are of -- subtype? Only about 7%
a = emdat_hydro[emdat_hydro['subtype']=='--']
print('losses from -- events / all losses = ', sum(a['tot_loss']) / sum(emdat_hydro['tot_loss']) *100, '%')

losses from -- events / all losses =  6.870115676221969 %


In [10]:
#number of hydro events without 'ISO' and the respective damage (1980-2010)
# Note: These events are accounted for later on extra

natcat_hydro_y = natcat_hydro[natcat_hydro['year'] <2011]

In [11]:
# create new columns in natcat_short and fill them with region names, region IDs and IDs from the ISO_codes.csv
# codes.csv is created in Regions_Floods_NATCAT

natcat_short = natcat_short.assign(Reg_name = np.NaN)
natcat_short = natcat_short.assign(ID = np.NaN)
natcat_short = natcat_short.assign(Reg_ID = np.NaN)


for i in range(0,len(list(num['ISO']))):
    if num['ISO'][i] in list(natcat_short['ISO']): # if ISO exists in all NatCat data  
        natcat_short.loc[natcat_short['ISO']==num['ISO'][i],'Reg_name' ] = num['Reg_name'][i]
        natcat_short.loc[natcat_short['ISO']==num['ISO'][i],'Reg_ID' ] = num['Reg_ID'][i]
        natcat_short.loc[natcat_short['ISO']==num['ISO'][i],'ID' ] = num['ID'][i]

# add also Reg names etc to natcat_hydro...        
natcat_hydro = natcat_hydro.assign(Reg_name = np.NaN)
natcat_hydro = natcat_hydro.assign(ID = np.NaN)
natcat_hydro = natcat_hydro.assign(Reg_ID = np.NaN)

for i in range(0,len(list(num['ISO']))):
    if num['ISO'][i] in list(natcat_hydro['ISO']): # if ISO exists in all NatCat data  
        natcat_hydro.loc[natcat_hydro['ISO']==num['ISO'][i],'Reg_name' ] = num['Reg_name'][i]
        natcat_hydro.loc[natcat_hydro['ISO']==num['ISO'][i],'Reg_ID' ] = num['Reg_ID'][i]
        natcat_hydro.loc[natcat_hydro['ISO']==num['ISO'][i],'ID' ] = num['ID'][i]
        

##add region name, ID etc to emdat        
for i in range(0,len(list(num['ISO']))):
    if num['ISO'][i] in list(emdat_hydro['ISO']):
        emdat_hydro.loc[emdat_hydro['ISO']==num['ISO'][i],'Reg_name' ] = num['Reg_name'][i]
        emdat_hydro.loc[emdat_hydro['ISO']==num['ISO'][i],'Reg_ID' ] = num['Reg_ID'][i]
        emdat_hydro.loc[emdat_hydro['ISO']==num['ISO'][i],'ID' ] = num['ID'][i]
        
        
#check -> works correctly
natcat_short.head()   

,year,event,type,subtype,ISO,country,Continent,tot_loss,ins_loss,tot_loss_GDP,tot_loss_GCP,tot_loss_CPI,Longitude,Latitude,Reg_name,ID,Reg_ID
0,1980,Drought,D,dr:Drought,ESP,Spain,EUROP,1500.00,0.0,7776.285421,10338.403680,3589.576050,-1.485377,38.002378,EUR,61.0,11.0
1,1980,Earthquake,A,gs:Ground shaking/Earthquake,PRT,Portugal,EUROP,40.00,0.0,258.086525,293.759557,118.148812,-27.220577,38.721642,EUR,169.0,11.0
2,1980,Cyclone Amy,B,tc:Tropical cyclone,AUS,Australia,OCEAN,4.00,3.0,30.554906,40.463144,10.901736,115.861000,-31.950000,AUS,13.0,10.0
3,1980,Snowstorms,B,bs:Blizzard/Snowstorm,AFG,Afghanistan,ASIA,0.01,0.0,0.050349,0.058855,0.029127,65.000000,33.000000,SWA,2.0,6.0
4,1980,Snowstorm,B,bs:Blizzard/Snowstorm,GRC,Greece,EUROP,0.10,0.0,0.328316,0.367235,0.263188,20.667000,39.167000,EUR,81.0,11.0


Calculate conversion factors

    convert_GDP: to convert losses based on 2016 GDP into absolut losses in 2005
    convert_CPI: to convert losses based on 2016 CPI into absolut losses in 2005



In [12]:
#Determine country list of conversion factors, for this we included ALL natural catastrophes 

# Note: we now use column 33 for CPI
natcat_convert=natcat.iloc[:,[4,7,8,9,13,12,11,21,20,25,29,33,46,47]] # same columns as for natcat_short
cols=['year','event','type','subtype','ISO','country','Continent','tot_loss','ins_loss','tot_loss_GDP','tot_loss_GCP','tot_loss_CPI', 'Longitude', 'Latitude']
natcat_convert.columns=cols

# Check:
natcat_convert.head()
natcat_convert[natcat_convert['year']==2005].head()
natcat_convert.head()

,year,event,type,subtype,ISO,country,Continent,tot_loss,ins_loss,tot_loss_GDP,tot_loss_GCP,tot_loss_CPI,Longitude,Latitude
0,1980,Drought,D,dr:Drought,ESP,Spain,EUROP,1500.00,0.0,7776.285421,10338.403680,3589.576050,-1.485377,38.002378
1,1980,Earthquake,A,gs:Ground shaking/Earthquake,PRT,Portugal,EUROP,40.00,0.0,258.086525,293.759557,118.148812,-27.220577,38.721642
2,1980,Cyclone Amy,B,tc:Tropical cyclone,AUS,Australia,OCEAN,4.00,3.0,30.554906,40.463144,10.901736,115.861000,-31.950000
3,1980,Snowstorms,B,bs:Blizzard/Snowstorm,AFG,Afghanistan,ASIA,0.01,0.0,0.050349,0.058855,0.029127,65.000000,33.000000
4,1980,Snowstorm,B,bs:Blizzard/Snowstorm,GRC,Greece,EUROP,0.10,0.0,0.328316,0.367235,0.263188,20.667000,39.167000


In [13]:
# Calculate conversion factors based on 2005 events

#make list: ISO, conversion factor
nat_cat_2005  = natcat_convert[natcat_convert['year']==2005] # only 2005 events
Reg_ID_2005   = natcat_short[natcat_short['year']==2005]['Reg_ID'] # list of region ids for 2005 events
Reg_name_2005 = natcat_short[natcat_short['year']==2005]['Reg_name'] # list of region names for 2005 events
ID_2005       = natcat_short[natcat_short['year']==2005]['ID'] # list of ids for 2005 events

#set up relevant columns
d = {'country':nat_cat_2005['country'],'ISO':nat_cat_2005['ISO'], 
     'convert_GDP':nat_cat_2005['tot_loss_GDP']/nat_cat_2005['tot_loss'],
     'convert_CPI':nat_cat_2005['tot_loss_CPI']/nat_cat_2005['tot_loss'],
     'Reg_name':Reg_name_2005,'Reg_ID':Reg_ID_2005,'ID':ID_2005,'year':natcat_short['year']}
#Make data frame
convert_na =  pd.DataFrame(data=d)
#drop rows containing 'NaN'
convert_zeros = convert_na.dropna(axis=0, how='any')
#drop rows where cnv == '0'
convert = convert_zeros[convert_zeros.convert_GDP!=0]

#save convert as .csv
convert.to_csv('conversions_inflation_2005.csv',index=None)

#Make list of countries that are not included in conversion list
missing_countries = set(num['ISO']) - set(convert['ISO']) 
available_countries = set(convert['ISO'])

#print missing_countries

#!!! CPI values differ for each events!! see line below
# This is because CPIs are determined sub-annually, always using the value of the last day of the month of the event
# except for 2016, when the last day of the year is taken
# Solution: Take yearly mean of CPI for conversion factors!

# Check:
print(convert[convert['ISO']=='JPN']['convert_CPI'])

16111    1.057408
16126    1.057408
16135    1.057408
16147    1.057408
16167    1.057408
16205    1.057408
16270    1.057408
16280    1.057408
16430    1.057408
16452    1.057408
16495    1.057408
16532    1.057408
16541    1.057408
16596    1.057408
16605    1.057408
16618    1.057408
16633    1.057408
16645    1.057408
16736    1.057408
16828    1.057408
16868    1.057408
Name: convert_CPI, dtype: float64


In [14]:
# CHECKCELL
nat_cat_2005.head()
convert.head()

,country,ISO,convert_GDP,convert_CPI,Reg_name,Reg_ID,ID,year
16023,Portugal,PRT,1.027892,1.038719,EUR,11.0,169.0,2005
16024,Spain,ESP,1.045679,1.067219,EUR,11.0,61.0,2005
16025,Portugal,PRT,1.027892,1.038719,EUR,11.0,169.0,2005
16026,Vietnam,VNM,3.542038,1.794471,SEA,7.0,223.0,2005
16027,Mozambique,MOZ,1.585458,0.750304,SSA,5.0,138.0,2005


In [17]:
# Calculate conversion factors for countries that are lacking events in 2005

########## For those countries that didn't experience any damage in 2005 find the closest pair 
##########of loss events around 2005 and determine their weighted mean

mc = list(missing_countries)
mmc =list() #list of countries for which a conversion factor can't be determined by extrapolating 

d = {'ISO':mc,'country':'nan','year_b':0.000,'year_a':0.000,
     'convert_GDP_a':0.000,'convert_CPI_a':0.000,
     'convert_GDP_b':0.000,'convert_CPI_b':0.000,
     'convert_GDP':0.000,'convert_CPI':0.000}

convert_mc = pd.DataFrame(data=d)

for i in range(0,len(mc)):
    indexes = [j for j,x in enumerate(natcat_convert['ISO']) if x == mc[i]] #find loss events of country c
    
    natcat_mc = natcat_convert.loc[indexes,:] #select all entries of countrie i
    natcat_mc = natcat_mc[natcat_mc['tot_loss'] !=0]         #remove all years with loss of 0
    natcat_before = natcat_mc[natcat_mc['year']<2005] #data.frame including all events before
    natcat_after = natcat_mc[natcat_mc['year']>2005]  #data.frame including all events after
    
    if len(natcat_before['tot_loss'])>0:  # we need at least two events
        if len(natcat_after['tot_loss'])>0:
            before = natcat_before.iloc[len(natcat_before['tot_loss'])-1] #get nearest event before
            after = natcat_after.iloc[0] #get nearest event after

            convert_GDP_b = before['tot_loss_GDP']/before['tot_loss']  #conversion factor before
            convert_GDP_a = after['tot_loss_GDP']/after['tot_loss'] 

            #print before 
            #print after 
            convert_CPI_b = before['tot_loss_CPI']/before['tot_loss'] #conversion factor after
            convert_CPI_a = after['tot_loss_CPI']/after['tot_loss'] 

    #determine weighting factors     
            d1 = 2005 - before['year']
            d2 = after['year'] - 2005
            d = d1 + d2
            w1 = d1/d
            w2 = d2/d
    #conversion factor is the sum of weighted factors before/after
            convert_CPI = w1*convert_CPI_b + w2*convert_CPI_a #determine mean conversion factor
            convert_GDP = w1*convert_GDP_b + w2*convert_GDP_a #determine mean conversion factor  

            #print convert_GDP
            #print convert_GDP_a
            #print convert_GDP_b

    #add conversion factor to list
            convert_mc.loc[i,'ISO'] = before['ISO']
            convert_mc.loc[i,'country'] = before['country']
            convert_mc.loc[i,'year_b'] =before['year']
            convert_mc.loc[i,'year_a'] =after['year']
            convert_mc.loc[i,'convert_GDP_b'] = convert_GDP_b
            convert_mc.loc[i,'convert_GDP_a'] = convert_GDP_a
            convert_mc.loc[i,'convert_CPI_b'] = convert_CPI_b
            convert_mc.loc[i,'convert_CPI_a'] = convert_CPI_a
            convert_mc.loc[i,'convert_GDP'] = convert_GDP
            convert_mc.loc[i,'convert_CPI'] = convert_CPI
    else: 
        
        mmc.append(mc[i])
     


In [18]:
# save to file
convert_mc = convert_mc[convert_mc['convert_CPI'] !=0]
convert_mc.to_csv('conversions_inflation_2005_missing_countries.csv',index=None)
convert_mc.head()

,ISO,country,year_b,year_a,convert_GDP_a,convert_CPI_a,convert_GDP_b,convert_CPI_b,convert_GDP,convert_CPI
0,BEN,Benin,1999.0,2007.0,1.495195,1.000927,3.320774,1.619995,2.864379,1.465228
1,BWA,Botswana,2004.0,2006.0,1.546415,1.035773,1.741322,1.008838,1.643868,1.022305
2,TUV,Tuvalu,1997.0,2011.0,0.835296,0.767948,2.453735,1.565111,1.760118,1.223470
3,SGP,Singapore,2004.0,2006.0,2.043445,1.480826,2.644727,1.597760,2.344086,1.539293
4,REU,Reunion,2002.0,2006.0,1.051740,0.991035,1.623679,1.419554,1.480694,1.312424


In [19]:
#Some countries have several damage events in 2005. While the GDP value is 
#determined on a yearly basis (same value for each event), the CPI shows different values
#per event. #This box determines the mean of the yearly CPI values per country and stores it in an array
 
#Calculate values for Suriname, use 2006 as reference period.
nat_cat_SUR = natcat_convert[natcat_convert['ISO']=='SUR']
SUR_CPI = nat_cat_SUR['tot_loss_CPI']/nat_cat_SUR['tot_loss']
SUR_GDP = nat_cat_SUR['tot_loss_GDP']/nat_cat_SUR['tot_loss']

print(SUR_CPI.iloc[0,])
print(SUR_GDP.iloc[0,])
    
d = {'ISO':num['ISO'],'convert_GDP':0.000,'convert_CPI':0.000}
convert_all = pd.DataFrame(data=d)

for ISO in list(num['ISO']):
    
    if ISO in list(available_countries):
        convert_all.loc[convert_all['ISO']==ISO,'convert_CPI'] = np.mean(convert.loc[convert['ISO']==ISO,'convert_CPI'])
        convert_all.loc[convert_all['ISO']==ISO,'convert_GDP'] = np.mean(convert.loc[convert['ISO']==ISO,'convert_GDP'])
    if ISO in list(missing_countries):
        convert_all.loc[convert_all['ISO']==ISO,'convert_CPI'] = np.mean(convert_mc.loc[convert_mc['ISO']==ISO,'convert_CPI'])
        convert_all.loc[convert_all['ISO']==ISO,'convert_GDP'] = np.mean(convert_mc.loc[convert_mc['ISO']==ISO,'convert_GDP'])
    if ISO == 'SUR': #take GDP and CPI conversion factor from 2006
        convert_all.loc[convert_all['ISO']==ISO,'convert_CPI'] = SUR_CPI.iloc[0,] #hardcoded values
        convert_all.loc[convert_all['ISO']==ISO,'convert_GDP'] = SUR_GDP.iloc[0,]

0.8331064452535
1.1803611060899


In [20]:
print ('There are ', convert_all['convert_CPI'].isnull().sum(), ' countries missing: \n', convert_all.loc[convert_all['convert_CPI'].isnull(), 'ISO'])

# Do these countries have floods at all? Only Lichtenstein has one...

natcat_short[natcat_short['ISO']=='LIE'] # yes, one
natcat_short[natcat_short['ISO']=='QAT'] # (only flash floods)
#natcat_short[natcat_short['ISO']=='STP'] 

There are  24  countries missing: 
 5      AND
6      ANT
65     FLK
72     GGY
74     GIB
79     GNQ
86     HKG
92     IMN
101    JEY
117    LIE
123    MAC
125    MCO
148    NFK
151    NIU
155    NRU
160    PCN
163    PLW
172    QAT
178    SCG
182    SHN
183    SJM
187    SMR
189    SPM
190    STP
Name: ISO, dtype: object


,year,event,type,subtype,ISO,country,Continent,tot_loss,ins_loss,tot_loss_GDP,tot_loss_GCP,tot_loss_CPI,Longitude,Latitude,Reg_name,ID,Reg_ID
28877,2015,Flash flood,C,ff:Flash flood,QAT,Qatar,ASIA,10.0,0.0,9.665693,10.077442,10.275497,51.522476,25.280282,ARA,173.0,15.0
30462,2016,Flash flood,C,ff:Flash flood,QAT,Qatar,ASIA,0.1,0.0,0.100000,0.100000,0.100000,51.522476,25.280282,ARA,173.0,15.0


In [21]:
#Add converted damages to entire natcat. 

natcat_short['CPI_conv'] = 0.0000 #add column to natcat_short that contains converted damages (2005)
natcat_short['GDP_conv'] = 0.0000 #add column to natcat_short



#Now the conversion of all remaining events, i.e., the events where ISOs exist
# this might be confusing, since this is the standard case which is taken care off in the end.
#Match 2005er countries/other countries: List all ISOs with conversion factor

#for i in range(0,len(list(natcat_short['ISO']))):
for i in range(0,len(list(convert_all['ISO']))):
        #print i
        indexes = [j for j,x in enumerate(natcat_short['ISO']) if x == list(convert_all['ISO'])[i]]
        natcat_short.loc[indexes,'CPI_conv'] = natcat_short['tot_loss_CPI'][indexes]/list(convert_all['convert_CPI'])[i]
        natcat_short.loc[indexes,'GDP_conv'] = natcat_short['tot_loss_GDP'][indexes]/list(convert_all['convert_GDP'])[i]

print(natcat_short.head())

   year        event type                       subtype  ISO      country  \
0  1980      Drought    D                    dr:Drought  ESP        Spain   
1  1980   Earthquake    A  gs:Ground shaking/Earthquake  PRT     Portugal   
2  1980  Cyclone Amy    B           tc:Tropical cyclone  AUS    Australia   
3  1980   Snowstorms    B         bs:Blizzard/Snowstorm  AFG  Afghanistan   
4  1980    Snowstorm    B         bs:Blizzard/Snowstorm  GRC       Greece   

  Continent  tot_loss  ins_loss  tot_loss_GDP  tot_loss_GCP  tot_loss_CPI  \
0     EUROP   1500.00       0.0   7776.285421  10338.403680   3589.576050   
1     EUROP     40.00       0.0    258.086525    293.759557    118.148812   
2     OCEAN      4.00       3.0     30.554906     40.463144     10.901736   
3      ASIA      0.01       0.0      0.050349      0.058855      0.029127   
4     EUROP      0.10       0.0      0.328316      0.367235      0.263188   

    Longitude   Latitude Reg_name     ID  Reg_ID     CPI_conv     GDP_conv

In [22]:
# Testcell: Do columns contain values?

print(natcat_short.loc[(natcat_short['ISO']=='DEU') & (natcat_short['year']==1980),:])

     year                  event type                       subtype  ISO  \
50   1980            Flash flood    C                ff:Flash flood  DEU   
51   1980           Severe storm    B       te:Tempest/Severe storm  DEU   
123  1980           Winter storm    B               ws:Winter storm  DEU   
127  1980             Earthquake    A  gs:Ground shaking/Earthquake  DEU   
130  1980          Winter damage    D              wd:Winter damage  DEU   
181  1980  Tornado, severe storm    B                    to:Tornado  DEU   
207  1980                Tornado    B                    to:Tornado  DEU   
213  1980                Tornado    B                    to:Tornado  DEU   
224  1980                Tornado    B                    to:Tornado  DEU   
226  1980             Earthquake    A  gs:Ground shaking/Earthquake  DEU   
252  1980            Flash flood    C                ff:Flash flood  DEU   
264  1980            Flash flood    C                ff:Flash flood  DEU   
287  1980   

In [23]:
## LOAD PPP CONVERSION AND MULTIPLY WITH THESE
# gap_filled version
# dataset
path='/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/PWT_based_converter_realGDP2005_to_PPPrealGDP2005-2_gap_filled.csv'
ppp_convert=pd.read_csv(path)

In [24]:
# CHECKBOX

print(ppp_convert.loc[(ppp_convert['ISO']=='DEU') & (ppp_convert['year']==2010)])
print(natcat_short.loc[(natcat_short['ISO']=='DEU') & (natcat_short['year']==2010),:])


      Unnamed: 0  Unnamed: 0.1  ISO  year  factor_to_ppp Reg_name
1342        1342        1342.0  DEU  2010       0.998038      EUR
       year                        event type                  subtype  ISO  \
21221  2010                Winter damage    D         wd:Winter damage  DEU   
21259  2010     Winter damage, snowstorm    D         wd:Winter damage  DEU   
21306  2010                     Rockfall    A              ro:Rockfall  DEU   
21314  2010  Winter storm, winter damage    B          ws:Winter storm  DEU   
21403  2010         Winter Storm Xynthia    B          ws:Winter storm  DEU   
21468  2010       Severe storms, tornado    B  te:Tempest/Severe storm  DEU   
21630  2010        Tornado, severe storm    B               to:Tornado  DEU   
21663  2010                       Floods    C         gf:General flood  DEU   
21713  2010                    Tornadoes    B               to:Tornado  DEU   
21719  2010                Severe storms    B  te:Tempest/Severe storm  DEU   

In [25]:
# Multiply events with -> pppfact (country ISO, year)


natcat_short['CPI_conv_ppp'] = 0.0000 #add column to natcat_short that contains ppp-converted damages (2005)
natcat_short['GDP_conv_ppp'] = 0.0000 #add column to natcat_short

for year in range(1980,2011):
    #print year
    for iso in list(convert_all['ISO']):
        #print iso
        convert_tmp = ppp_convert.loc[(ppp_convert['ISO']==iso) & (ppp_convert['year']==year), 'factor_to_ppp']
        if convert_tmp.empty==False:
            natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'CPI_conv_ppp'] = np.array(convert_tmp)[0] * natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'CPI_conv']
            natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'GDP_conv_ppp'] = np.array(convert_tmp)[0] * natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'GDP_conv']
        else:
            natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'CPI_conv_ppp'] = 1 * natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'CPI_conv']
            natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'GDP_conv_ppp'] = 1 * natcat_short.loc[(natcat_short['ISO']==iso) & (natcat_short['year']==year), 'GDP_conv']

In [26]:
# CHECKBOX

year = 1980
iso = 'DEU'
print(ppp_convert.loc[(ppp_convert['ISO']=='DEU') & (ppp_convert['year']==1980)])

print(natcat_short.loc[(natcat_short['ISO']=='DEU') & (natcat_short['year']==1980),:])

# Sieht korrekt aus...

      Unnamed: 0  Unnamed: 0.1  ISO  year  factor_to_ppp Reg_name
1312        1312        1312.0  DEU  1980       0.784224      EUR
     year                  event type                       subtype  ISO  \
50   1980            Flash flood    C                ff:Flash flood  DEU   
51   1980           Severe storm    B       te:Tempest/Severe storm  DEU   
123  1980           Winter storm    B               ws:Winter storm  DEU   
127  1980             Earthquake    A  gs:Ground shaking/Earthquake  DEU   
130  1980          Winter damage    D              wd:Winter damage  DEU   
181  1980  Tornado, severe storm    B                    to:Tornado  DEU   
207  1980                Tornado    B                    to:Tornado  DEU   
213  1980                Tornado    B                    to:Tornado  DEU   
224  1980                Tornado    B                    to:Tornado  DEU   
226  1980             Earthquake    A  gs:Ground shaking/Earthquake  DEU   
252  1980            Flash flood

In [27]:
###Print Natcat_data with CPI_2005 and GDP_2005 into .xls file

#print natcat_short.head()
#print natcat.head()
#print len(natcat_short)
#print len(natcat)

natcat['CPI_con'] = natcat_short['CPI_conv_ppp'] # include ppp-conversion
natcat['GDP_con'] = natcat_short['GDP_conv_ppp'] # include ppp-conversion
natcat['Reg_name'] = natcat_short['Reg_name']

writer = pd.ExcelWriter('/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/1980-2016_Masterdatei_NatCat_worldwide_no-pw_2005conversions_PPP.xlsx')
natcat.to_excel(writer)
writer.save()

print('Added damages converted to 2005 values based on GDP and CPI to the NatCat data, includes ppp conversion')

Added damages converted to 2005 values based on GDP and CPI to the NatCat data, includes ppp conversion


In [28]:
natcat_set = set(list(natcat_hydro_y['ISO']))
convert_set = set(list(ppp_convert['ISO']))

in_nat_cat = list(natcat_set & set(natcat_set ^ convert_set))
in_nat_cat

[nan, 'GRL', 'ATA']

In [29]:
in_conv = list(natcat_set & set(natcat_set ^ convert_set))
in_conv

[nan, 'GRL', 'ATA']

In [122]:
# try to fill this data with conversion factors from WB
# test Germany
wb_conv_factors = pd.read_csv('/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/WB_conversion_factors.csv',na_values ='..')
wb_conv_factors.columns = ['Series Name', 'Series Code', 'Country Name', 'ISO', '1980','1981','1982','1983', '1984',
                          '1985','1986','1987','1988','1989','1990','1991','1992','1993','1994','1995','1996','1997',
                          '1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
                          '2011','2012','2013','2014','2015','2016','2017']
wb_conv_factors = wb_conv_factors[wb_conv_factors['Series Code']=='PA.NUS.PPP']
ger_compare = ppp_convert[ppp_convert['ISO']=='DEU']
test= wb_conv_factors.loc[wb_conv_factors['ISO']=='DEU',:]

ger_compare['WB_factors'] = test.iloc[:,4:36].values[0]

ger_compare['USA_conv'] = np.array(ppp_convert.loc[(ppp_convert['ISO']=='USA')& (ppp_convert['year']>=1980)&(ppp_convert['year']<=2012),'factor_to_ppp'])
ger_compare['test_comp']= ger_compare['USA_conv']
ger_compare

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,Unnamed: 0,ISO,year,factor_to_ppp,WB_factors,USA_conv,test_comp
1312,1312,DEU,1980,0.784224,NaN,1.008800,1.008800
1313,1313,DEU,1981,0.765460,NaN,1.009369,1.009369
1314,1314,DEU,1982,0.751256,NaN,1.003093,1.003093
1315,1315,DEU,1983,0.739775,NaN,1.002715,1.002715
1316,1316,DEU,1984,0.727343,NaN,1.007570,1.007570
1317,1317,DEU,1985,0.715222,NaN,1.001080,1.001080
1318,1318,DEU,1986,0.759174,NaN,0.997330,0.997330
1319,1319,DEU,1987,0.785771,NaN,0.997276,0.997276
1320,1320,DEU,1988,0.790034,NaN,0.996636,0.996636
1321,1321,DEU,1989,0.790563,NaN,0.996433,0.996433


In [133]:
# use average conversion-factor for the region to fill gaps

data_frame = pd.read_csv('/home/insauer/Climada/climada_python/data/system/NatRegIDs.csv')
regs = set(list(data_frame['Reg_name']))
cntr = set(list(ppp_convert['ISO']))

#missing_countries.remove('nan')
#missing_countries.remove('SYC')
for cnt in cntr:
    if np.isin(cnt, data_frame['ISO']):
        reg = data_frame.loc[data_frame['ISO']==cnt, 'Reg_name'].values[0]
        ppp_convert.loc[ppp_convert['ISO']== cnt, 'Reg_name'] = reg
ppp_convert.loc[ppp_convert['Reg_name']=='PIS2']

,Unnamed: 0,ISO,year,factor_to_ppp,Reg_name


In [124]:
def aggregation_regions(x):
    reg_factor_ppp = x['factor_to_ppp'].mean()
    return pd.Series([reg_factor_ppp], index=['factor_to_ppp'])


In [125]:
ppp_aggr = ppp_convert.groupby(['year', 'Reg_name'])[['factor_to_ppp']].apply(aggregation_regions).reset_index()
ppp_aggr.to_csv('/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/PWT_based_converter_realGDP2005_to_PPPrealGDP2005-2_regs.csv')

In [128]:
missing_countries = in_conv[1:]
missing_countries.remove('ATA')
missing_countries.remove('GRL')
missing_countries

['COK',
 'GLP',
 'MTQ',
 'MHL',
 'MMR',
 'CUB',
 'PRK',
 'PYF',
 'HTI',
 'TLS',
 'PNG',
 'GUY',
 'SLB',
 'ARE',
 'LIE',
 'SOM',
 'PSE',
 'ERI',
 'VUT',
 'AFG',
 'ESH',
 'NCL',
 'SSD',
 'AND',
 'GUM',
 'SYC',
 'NIC',
 'DZA',
 'PRI']

In [136]:
for mc in missing_countries:
    if (mc =='ESH'):
        r = 'NAF'
    elif mc == 'SSD':
        r = 'SSA'
    elif mc == 'SYC':
        r = 'SSA'
    else:
        r = data_frame.loc[data_frame['ISO']==mc,'Reg_name'].values[0]
    print(r)
    if r == 'PIS2':
        rf='PIS1'
    else:
        rf = r
    for y in range(1980,2012):
        reg_conv = ppp_aggr.loc[(ppp_aggr['year']==y)&(ppp_aggr['Reg_name']==rf),'factor_to_ppp'].values[0]
        ppp_convert = ppp_convert.append({'ISO': mc, 'year': y, 'factor_to_ppp': reg_conv, 'Reg_name': r}, ignore_index=True)

PIS2
CAR
CAR
PIS1
SWA
CAR
CHN
PIS2
CAR
SEA
SEA
LAN
PIS1
ARA
EUR
SSA
ARA
SSA
PIS1
SWA
NAF
PIS1
SSA
EUR
PIS1
SSA
CAR
NAF
CAR


In [138]:
ppp_convert.to_csv('/home/insauer/projects/Attribution/Floods/Paper_NC_Review_Data/Input_PPP_conversion/PWT_based_converter_realGDP2005_to_PPPrealGDP2005-2_gap_filled.csv')

In [147]:
natcat

,Event ID,MR Number,MR ID,Period,Year,Begin,End,Event Text,Event Family \n(A = Geophysical events; \nB = Meteorological events; \nC = Hydrological events; \nD = Climatological/Extreme temperature events),Subevent,...,"official overall losses for publication\n(US$ m, normalized to 2016 values based on GCP method)","official insured losses for publication\n(US$ m, normalized to 2016 values based on GCP method, change in insurance density not considered)","estimated overall losses for statistics\n(US$ m, normalized to 2016 values based on GCP method)","estimated insured losses for statistics\n(US$ m, normalized to 2016 values based on GCP method, change in insurance density not considered)","official overall losses for publication\n(US$ m, adjusted to inflation to 2016 values based on national CPI)","official insured losses for publication\n(US$ m, adjusted to inflation to 2016 values based on national CPI)","estimated overall losses for statistics\n(US$ m, adjusted to inflation to 2016 values based on national CPI)",CPI_con,GDP_con,Reg_name
0,15344,MR198001D066,MR198001D066_ESP,January 1980- August 1982,1980,1980-01-01,1982-08-31,Drought,D,dr:Drought,...,10338.403680,0.000000,10338.403680,0.000000,3589.576050,0.000000,3589.576050,2879.544250,6366.606950,EUR
1,14808,MR198001A033,MR198001A033_PRT,1.1.1980,1980,1980-01-01,1980-01-01,Earthquake,A,gs:Ground shaking/Earthquake,...,293.759557,0.000000,293.759557,0.000000,118.148812,0.000000,118.148812,90.931928,200.725745,EUR
2,10844,MR198001B003,MR198001B003_AUS,January 1980,1980,1980-01-01,1980-01-01,Cyclone Amy,B,tc:Tropical cyclone,...,40.463144,30.347358,40.463144,30.347358,10.901736,8.176302,10.901736,7.494590,16.013607,AUS
3,12372,MR198001B005,MR198001B005_AFG,Januar 1980,1980,1980-01-01,1980-01-31,Snowstorms,B,bs:Blizzard/Snowstorm,...,0.000000,0.000000,0.058855,0.000000,0.000000,0.000000,0.029127,0.027904,0.021407,SWA
4,13659,MR198001B022,MR198001B022_GRC,1.-3.1.1980,1980,1980-01-01,1980-01-03,Snowstorm,B,bs:Blizzard/Snowstorm,...,0.000000,0.000000,0.367235,0.000000,0.000000,0.000000,0.263188,0.192064,0.326273,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30557,14806040,MR201612C444,MR201612C444_ECU,29.-30.12.2016,2016,2016-12-29,2016-12-30,"Flash flood, landslide",C,ff:Flash flood,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,LAN
30558,14794827,MR201612B442,MR201612B442,29.-30.12.2016,2016,2016-12-29,2016-12-30,Winter storm,B,ws:Winter storm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
30559,14735291,MR201612B442,MR201612B442_CAN,30.12.2016,2016,2016-12-30,2016-12-30,Winter storm,B,ws:Winter storm,...,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,NAM
30560,14733806,MR201612C425,MR201612C425_BOL,30.12.2016,2016,2016-12-30,2016-12-30,"Flash flood, landslide",C,ff:Flash flood,...,0.000000,0.000000,1.500000,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,LAN
